## Initial Process

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Import CSV Data

In [ ]:
amnist_data = pd.read_csv("MNIST_train.csv")


In [ ]:
amnist_data

In [ ]:
amnist_data.shape

In [ ]:
X = amnist_data.to_numpy()

In [ ]:
y = X[:,2]
y

In [ ]:
set(y)

In [ ]:
X = X[:, 3:]
X.shape

In [ ]:
obs_1 = X[1, :]
obs_1

In [ ]:
X[1].max()

In [ ]:
obs_1 = obs_1.reshape(28, 28)
obs_1

In [ ]:
plt.imshow(obs_1)

In [ ]:
from scipy.stats import multivariate_normal as mvn

In [ ]:
class GaussNB():

  def fit(self, X, y, epsilon = 1e-3):
    self.likelihoods = dict()
    self.priors = dict()
    self.K = set(y.astype(int))

    for k in self.K:
      X_k = X[y==k]
      # Naive Assumption: Observations are linearly independant of each other
      self.likelihoods[k] = {"mean": X_k.mean(axis=0), "cov":X_k.var(axis=0)+epsilon}
      self.priors[k] = len(X_k) / len(X)

  def predict(self, X):

    N, D = X.shape
    P_hat = np.zeros((N, len(self.K)))

    for k, l in self.likelihoods.items():
      P_hat[:,k] = mvn.logpdf(X, l["mean"], l["cov"]) + np.log(self.priors[k])

    return P_hat.argmax(axis=1)

In [ ]:
gnb = GaussNB()
gnb.fit(X,y)
y_hat = gnb.predict(X)

In [ ]:
def accuracy(y, y_hat):
  return np.mean(y==y_hat)

In [ ]:
plt.figure()
plt.scatter(X[:,0], X[:, 1], c=y, alpha=0.4, s=10)

In [ ]:
accuracy(y, y_hat)